In [1]:
import sys, os
sys.path.append("../..")

# import gempy
import gempy as gp

# inline figures in jupyter notebooks
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pn

../..\gempy\visualization.py:31: UserWarning: Vtk package is not installed. No vtk visualization available.
  warnings.warn('Vtk package is not installed. No vtk visualization available.')
../..\gempy\posterior_analysis.py:24: UserWarning: pymc (v2) package is not installed. No support for stochastic simulation posterior analysis.
  warnings.warn("pymc (v2) package is not installed. No support for stochastic simulation posterior analysis.")


In [2]:
import gempy.geomodeller_integration as gi


In [3]:
gi

<module 'gempy.geomodeller_integration' from '../..\\gempy\\geomodeller_integration.py'>

In [5]:
geoM = gi.GeomodellerClass()

In [7]:
geoM.load_geomodeller_file(r'C:\Users\legui\PycharmProjects\gempy\test\GeoModeller\test_a\test_a.xml')

In [8]:
geoM.get_sections()

[<Element '{http://www.geomodeller.com/geo}Section' at 0x00000295953089A8>,
 <Element '{http://www.geomodeller.com/geo}Section' at 0x00000295952D68B8>]

In [9]:
geoM.get_faults()

No faults found in model


In [10]:
geoM.get_formations()

In [12]:
geoM.formations

[<Element '{http://www.geomodeller.com/geo}Formation' at 0x00000295952F1048>,
 <Element '{http://www.geomodeller.com/geo}Formation' at 0x00000295952F1AE8>]

In [27]:
geoM.get_formation_names()

['A', 'B']

In [14]:
geoM.get_stratigraphy_list()

['B', 'A', 'out']

In [24]:
geoM.get_provenance(geoM.get_sections()[0])

In [26]:
geoM.get_section_names()

['Middle', 'SurfaceTopography']

In [28]:
geoM.get_points_in_sections()


ObsID = None
	Formation name	= A
	X = 2.0, Y = -2.0
	X = 8.0, Y = -2.0

ObsID = None
	Formation name	= B
	X = 2.0, Y = -8.0
	X = 8.0, Y = -8.0


--------------------------------------------------------------------------------
Foliations in section Middle:
--------------------------------------------------------------------------------

ObsID = 
	Formation name	= A
	Azimuth = 90.0, dip = 0.0
	X = 5.0, Y = -5.0
			No Formation Points in this section


--------------------------------------------------------------------------------
Foliations in section SurfaceTopography:
--------------------------------------------------------------------------------
			No foliations in this section


In [30]:
geoM.get_drillhole_elements()

Problem with drillhole element; check if drillholes are defined in project!


In [32]:
geoM.create_formation_dict()
geoM.formation_dict

{'A': <Element '{http://www.geomodeller.com/geo}Formation' at 0x00000295952F1048>,
 'B': <Element '{http://www.geomodeller.com/geo}Formation' at 0x00000295952F1AE8>}

In [42]:
geoM.get_foliations(geoM.get_sections()[0])

[<Element '{http://www.geomodeller.com/geo}Foliation' at 0x00000295952D63B8>]

In [35]:
a = geoM.get_sections()[0]

In [44]:
geoM.get_foliation_coordinates(geoM.get_foliations(geoM.get_sections()[0])[0])

'5.0,-5.0'

In [45]:
geoM.get_formation_point_data(geoM.get_sections()[0])

[<Element '{http://www.geomodeller.com/geo}Interface' at 0x00000295952D6048>,
 <Element '{http://www.geomodeller.com/geo}Interface' at 0x00000295952D6188>]

In [46]:
geoM.get_point_coordinates(geoM.get_formation_point_data(geoM.get_sections()[0]))

['A 2,-2 8,-2 ', 'B 2,-8 8,-8 ']